In [1]:
from tkinter import Tk, Button
from tkinter.font import Font
from copy import deepcopy
import time

# Constants
size = 3
human_player = 'X'
computer_player = 'O'
empty = ' '
max_util = +1000
min_util = -1000

class State:
    def __init__(self, next_player, other=None):
        self.next_player = next_player
        self.table = {}
        self.value = 0
        
        for y in range(size):
            for x in range(size):
                self.table[x, y] = empty

        if other:
            self.__dict__ = deepcopy(other.__dict__)

    def is_full(self):
        return all(self.table[x, y] != empty for x in range(size) for y in range(size))

    def won(self, player):
        # Check rows
        for y in range(size):
            if all(self.table[x, y] == player for x in range(size)):
                return True
        # Check columns
        for x in range(size):
            if all(self.table[x, y] == player for y in range(size)):
                return True
        # Check diagonals
        if all(self.table[i, i] == player for i in range(size)):
            return True
        if all(self.table[i, size-1-i] == player for i in range(size)):
            return True
        return False

def actions(state):
    children = []
    for x in range(size):
        for y in range(size):
            if state.table[x, y] == empty:
                new_state = State(state.next_player, state)
                new_state.table[x, y] = state.next_player
                new_state.next_player = computer_player if state.next_player == human_player else human_player
                children.append(new_state)
    return children

def terminal_test(state):
    return state.won(human_player) or state.won(computer_player) or state.is_full()

def utility(state):
    if state.won(computer_player):
        return max_util
    elif state.won(human_player):
        return min_util
    return 0

def minimax(state, depth, alpha, beta, maximizing_player):
    if terminal_test(state) or depth == 0:
        return heuristic(state)
    
    if maximizing_player:
        max_eval = min_util
        for child in actions(state):
            eval = minimax(child, depth-1, alpha, beta, False)
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = max_util
        for child in actions(state):
            eval = minimax(child, depth-1, alpha, beta, True)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval

def heuristic(state):
    score = 0
    # Evaluate rows
    for y in range(size):
        line = [state.table[x, y] for x in range(size)]
        score += evaluate_line(line)
    # Evaluate columns
    for x in range(size):
        line = [state.table[x, y] for y in range(size)]
        score += evaluate_line(line)
    # Evaluate diagonals
    diag1 = [state.table[i, i] for i in range(size)]
    diag2 = [state.table[i, size-1-i] for i in range(size)]
    score += evaluate_line(diag1) + evaluate_line(diag2)
    return score

def evaluate_line(line):
    if line.count(computer_player) == 3: return 100
    if line.count(computer_player) == 2 and line.count(empty) == 1: return 10
    if line.count(computer_player) == 1 and line.count(empty) == 2: return 1
    if line.count(human_player) == 3: return -100
    if line.count(human_player) == 2 and line.count(empty) == 1: return -10
    if line.count(human_player) == 1 and line.count(empty) == 2: return -1
    return 0

def find_best_move(state):
    best_move = None
    best_value = min_util
    for move in actions(state):
        move_value = minimax(move, 5, min_util, max_util, False)
        if move_value > best_value:
            best_value = move_value
            best_move = move
    return best_move

class GameGUI:
    def __init__(self):
        self.game = State(human_player)
        self.app = Tk()
        self.app.title('Tic Tac Toe')
        self.font = Font(family="Helvetica", size=32)
        self.buttons = {}
        
        for x in range(size):
            for y in range(size):
                handler = lambda x=x, y=y: self.human_move(x, y)
                button = Button(self.app, command=handler, font=self.font, width=2, height=1)
                button.grid(row=x, column=y)
                self.buttons[x, y] = button
        
        reset_button = Button(self.app, text='Reset', command=self.reset)
        reset_button.grid(row=size, column=0, columnspan=size, sticky='WE')
        
        self.update()
    
    def reset(self):
        self.game = State(human_player)
        self.update()
    
    def human_move(self, x, y):
        if self.game.table[x, y] != empty or terminal_test(self.game):
            return
            
        self.game.table[x, y] = human_player
        self.game.next_player = computer_player
        self.update()
        
        if not terminal_test(self.game):
            self.app.after(100, self.computer_move)
    
    def computer_move(self):
        self.game = find_best_move(self.game)
        self.update()
    
    def update(self):
        for (x, y) in self.game.table:
            text = self.game.table[x, y]
            self.buttons[x, y]['text'] = text
            self.buttons[x, y]['disabledforeground'] = 'black'
            self.buttons[x, y]['state'] = 'normal' if text == empty else 'disabled'
        
        if terminal_test(self.game):
            winner = computer_player if self.game.won(computer_player) else human_player if self.game.won(human_player) else None
            if winner:
                color = 'red' if winner == human_player else 'blue'
                for (x, y) in self.buttons:
                    self.buttons[x, y]['disabledforeground'] = color
            for (x, y) in self.buttons:
                self.buttons[x, y]['state'] = 'disabled'
    
    def run(self):
        self.app.mainloop()

if __name__ == "__main__":
    GameGUI().run()